In [35]:

import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
from dotenv import load_dotenv
import bs4
from langchain_groq import ChatGroq

load_dotenv()

## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']

In [31]:

embeddings = OllamaEmbeddings()

loader=WebBaseLoader(web_paths=("https://www.taniarascia.com/getting-started-with-react/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs[:50])
vectors = FAISS.from_documents(final_documents, embeddings)

In [42]:

llm=ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")



prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = st.session_state.vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


response=retrieval_chain.invoke({"input":'create an objective question from the context given. question should have 4 options out of which 1 is correct'})
print(response['answer'])

KeyboardInterrupt: 